In [1]:
from itertools import count
import os
import gc
import time
import pandas as pd
import numpy as np
import zmq
import DRL_pb2
import time
import math
import visdom

from agent.main import Agent
from agent.actorcritic import OUNoise
from emulator.main4test import Account

from IPython.display import clear_output

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from params_bt import *
Instrument_ID = 'TA'

DRL = DRL_pb2.Agent_Info()
context = zmq.Context()
socket = context.socket(zmq.SUB)
IPC_CHANNEL = 'ipc:///tmp/' + Instrument_ID +'DRL'
socket.connect(IPC_CHANNEL)
socket.setsockopt_string(zmq.SUBSCRIBE,'')

os.system('rm /AI/ddpg_exp/model/'+Instrument_ID +'/*')
os.system('rm /AI/ddpg_exp/figure/'+Instrument_ID +'/*')

TD = '2019-03-01'
SD = '2018-12-01'

import per


In [ ]:
last_sharpe = 0
i = 1000
best_reward = 0
best_sharpe = 0

ou_noise = OUNoise(ACTION_SIZE)
env = Account(ST_D, ED_D,Instrument_ID)
agent_t = Agent(env.reset().shape, ACTION_SIZE)
Test_Full_Size = env.data_observation.shape[0] - 1

validation_env = Account(ST_D, TD,Instrument_ID)
Validattion_Full_Size = validation_env.data_observation.shape[0] - 1

viz_best = visdom.Visdom(env = Instrument_ID + '_Showing_Best')
viz_best.close()
BEST_IMG = viz_best.line(np.arange(20))

while True:
    r_msg = socket.recv()
    DRL.ParseFromString(r_msg)
    
    if DRL.episode == 0:
        last_sharpe = 0
    
    state = env.reset()
    
    mod_no = 9999 + math.sin(math.pi/2 * (DRL.episode % 2))
    try:                                      
        agent_t.load_model(Instrument_ID, mod_no)
    except:
        continue
    
    for _ in range(Test_Full_Size):
        action = agent_t.get_exploration_policy(np.expand_dims(state, 0))
        next_state, reward, done = env.step(action)
        state = next_state
        
    tmp = env.plot_data()
    sharpe = np.mean(tmp['reward'])/np.std(tmp['reward'])
    print(sharpe)
    
    if sharpe > last_sharpe * 0.9 and env.total_value > 1e6:
        clear_output(True)
        agent_t.save_model(Instrument_ID, i)
        last_sharpe = sharpe
        
        tmp.iloc[:, 0].plot(figsize=(16, 6))
        pic_name = './figure/'+ Instrument_ID + '/样本外' + str(i) + '.png'
        i = i + 1
        
        if os.path.isfile(pic_name):
            os.remove(pic_name)
        plt.savefig(pic_name)
        plt.close()
        
        ratio = np.sum(np.maximum(np.sign(tmp['reward']), 0))/tmp.shape[0]
        print("胜率:", ratio)
        print("sharpe:", sharpe)
        print(time.clock())
        
        ################################################################
        v_state = validation_env.reset()
        for _ in range(Validattion_Full_Size):
            action = agent_t.get_exploration_policy(np.expand_dims(v_state, 0))
            v_next_state, v_reward, _ = validation_env.step(action)
            v_state = v_next_state
        
        if validation_env.total_value > best_reward:
            best_sharpe = sharpe
            best_reward = validation_env.total_value
            agent_t.save_model(Instrument_ID, '_best')
            showing_rewards = validation_env.plot_data()['value']
            for _ in range(showing_rewards.size):
                if _ == 0:
                    viz_best.line(Y = np.array([showing_rewards[_]]),   X = np.array([_]), update = 'replace', win = BEST_IMG,\
                  opts = dict(title = str(i)))
                else:
                    viz_best.line(Y = np.array([showing_rewards[_]]),   X = np.array([_]), update = 'append', win = BEST_IMG,\
                  opts = dict(title = str(i)))

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


-0.007752155666106862
-0.007752263858240885
-0.0077523535529842105
-0.007752162447248032
-0.0077519967100586114
-0.007751929834495731
-0.007751894603990917
